In [40]:
import numpy as np
import cv2
import pandas as pd
import os
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt


from keras.models import Model
from keras.models import model_from_json
from keras.models import load_model
# from keras.preprocessing import image
import keras.utils as image
# from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

from scipy import spatial


%matplotlib inline

In [3]:
path = '/kaggle/input/video0/frame0/'

In [9]:
data = pd.read_csv(os.path.join(path, 'video0_labels.csv'))

In [11]:
df = data[data['frame_number']==0]
df.head(3)

,frame_number,object_id,label,x,y,w,h
0,0,8,0,0.4906,0.3667,0.3188,0.7278
1,0,23,2,0.1016,0.2250,0.1250,0.4444
2,0,28,2,0.0430,0.4972,0.0859,0.3278


In [37]:
def load_model(model_name, include_top=True):
    """ Load pre-trained Keras model
    Args:
        model_name: String, name of model to load
        include_top: String, the model is buildt with 'feature learning block' + 'classification block'
    Returns:
        model: Keras model instance
    """
    if selected_model in available_models:
        # Load a Keras instance
        try:
            if model_name == 'vgg16':
                model = VGG16(weights='imagenet', include_top=include_top)
            elif model_name == 'resnet50':
                model = ResNet50(weights='imagenet', include_top=include_top)
            print(f">> '{model.name}' model successfully loaded!")
        except:
            print(f">> Error while loading model '{selected_model}'")
    
    # Wrong selected model
    else:
        print(f">> Error: there is no '{selected_model}' in {available_models}")
    
    return model


def get_img_size_model(model):
    """Returns image size for image processing to be used in the model
    Args:
        model: Keras model instance 
    Returns:
        img_size_model: Tuple of integers, image size
    """
    model_name = model.name
    if model_name == "vgg16":
        img_size_model = (224, 224)
    elif model_name == "resnet50":
        img_size_model = (224, 224)
    else:
        img_size_model = (224, 224)
        print("Warning: model name unknown. Default image size: {}".format(img_size_model))
        
    return img_size_model


def image_processing(img_array):
    """ Preprocess image to be used in a keras model instance
    Args:
        img_array: Numpy array of an image which will be predicte
    Returns:
        processed_img = Numpy array which represents the processed image
    """    
    # Expand the shape
    img = np.expand_dims(img_array, axis=0)

    # Convert image from RGB to BGR (each color channel is zero-centered with respect to the ImageNet dataset, without scaling)
    processed_img = preprocess_input(img)
    
    return processed_img



def get_layername_feature_extraction(model):
    """ Return the name of last layer for feature extraction   
    Args:
        model: Keras model instance
    Returns:
        layername_feature_extraction: String, name of the layer for feature extraction
    """
    model_name = model.name
    if model_name == "vgg16":
        layername_feature_extraction = 'fc2'
    elif model_name == "resnet50":
        layername_feature_extraction = 'predictions'
    else:
        layername_feature_extraction = ''
        print("Warning: model name unknown. Default layername: '{}'".format(layername_feature_extraction))
    
    return layername_feature_extraction


def get_feature_vector(model, img_path):
    """ Get a feature vector extraction from an image by using a keras model instance
    Args:
        model: Keras model instance used to do the classification.
        img_path: String to the image path which will be predicted
    Returns:
        feature_vect: List of visual feature from the input image
    """
    
    # Creation of a new keras model instance without the last layer
    layername_feature_extraction = get_layername_feature_extraction(model)
    model_feature_vect = Model(inputs=model.input, outputs=model.get_layer(layername_feature_extraction).output)
    
    # Image processing
    img_size_model = get_img_size_model(model)
    img = image.load_img(img_path, target_size=img_size_model)
    img_arr = np.array(img)
    img_ = image_processing(img_arr)
    
    # Visual feature extraction
    feature_vect = model_feature_vect.predict(img_)
    
    return feature_vect



def calculate_similarity(vector1, vector2):
    """Compute similarities between two images using 'cosine similarities'
    Args:
        vector1: Numpy vector to represent feature extracted vector from image 1
        vector2: Numpy vector to represent feature extracted vector from image 1
    Returns:
        sim_cos: Float to describe the similarity between both images
    """
    sim_cos = 1-spatial.distance.cosine(vector1, vector2)
    
    return sim_cos



def compute_similarity_img(model, img_path_1, img_path_2):
    """ Return a cosine similarity between both images and display them in HTML
    Args:
        model: Keras model instance used to do the feature extraction
        img_path_1: String to the image 1 path
        img_path_2: String to the image 2 path
    Returns:
        sim_cos: Float to describe the similarity between both images
    """
    filename1 = os.path.basename(img_path_1).split(".")[0]
    filename2 = os.path.basename(img_path_2).split(".")[0]
    
    # Compute feature vector extracted
    fea_vec_img1 = get_feature_vector(model, img_path_1)
    fea_vec_img2 = get_feature_vector(model, img_path_2)
    
    # Compute cosine similarity
    sim_cos = calculate_similarity(fea_vec_img1, fea_vec_img2)
    
    # Read images
    img_size_model = get_img_size_model(model)
    im1 = cv2.resize(cv2.imread(img_path_1), dsize=img_size_model, interpolation = cv2.INTER_AREA)
    im2 = cv2.resize(cv2.imread(img_path_2), dsize=img_size_model, interpolation = cv2.INTER_AREA)
    
    # Concatenate images horizontally
    im12 = cv2.hconcat([im1, im2])
    
    # Save concatenated image
    dst_dir_cos_sim = "../report/cos_sim"
    create_folder(dst_dir_cos_sim)
    dst_dir = f"{dst_dir_cos_sim}/{model.name}"
    create_folder(dst_dir)
    
    new_filename = f"{filename1}_{filename2}"
    cv2.imwrite(f"{dst_dir}/{new_filename}.jpg", im12)

    # Display images with cosine similarity result with HTML
    thumb = "<div style='margin: 0px;'> ({}) {}/{}: cos_sim = {:.4f}</div>".format(model.name, filename1, filename2, sim_cos)
    thumb += ''.join( ["<img style='width: 300px; margin: 0px; float: left; border: 1px solid black;' src='%s'/>" %str(s)
                       for s in sorted(glob('{}/{}.jpg'.format(dst_dir, new_filename))) ])
    
    display(HTML(thumb))
    print()
    
    return sim_cos


In [38]:
available_models = ['vgg16', 'resnet50']
selected_model = 'resnet50'

model = load_model(selected_model, include_top=True)

>> 'resnet50' model successfully loaded!


In [41]:
img1 = os.path.join(path, '8.jpg')
img2 = os.path.join(path, '213.jpg')

compute_similarity_img(model, img1, img2)

1/1 [==============================] - 1s 846ms/step


ValueError: Input vector should be 1-D.